# Installations

In [1]:
!pip3 install pandas
!pip3 install numpy
!pip3 install seaborn
!pip3 install scikit-learn
!pip3 install matplotlib

# Imports

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# EDA
We use dataset for credit score classification: https://www.kaggle.com/datasets/parisrohan/credit-score-classification/data

In [3]:
df = pd.read_csv('data/train.csv')
print(f"Dataset length: {len(df)}")

Dataset length: 100000


/var/folders/9z/rhfc541d14q5klxp9lqvgx8m0000gq/T/ipykernel_94310/1170968565.py:1: DtypeWarning: Columns (26) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('data/train.csv')


In [4]:
df.columns = df.columns.str.lower()

In [5]:
df.head()

,id,customer_id,month,name,age,ssn,occupation,annual_income,monthly_inhand_salary,num_bank_accounts,...,credit_mix,outstanding_debt,credit_utilization_ratio,credit_history_age,payment_of_min_amount,total_emi_per_month,amount_invested_monthly,payment_behaviour,monthly_balance,credit_score
0,0x1602,CUS_0xd40,January,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,1824.843333,3,...,_,809.98,26.822620,22 Years and 1 Months,No,49.574949,80.41529543900253,High_spent_Small_value_payments,312.49408867943663,Good
1,0x1603,CUS_0xd40,February,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,NaN,3,...,Good,809.98,31.944960,NaN,No,49.574949,118.28022162236736,Low_spent_Large_value_payments,284.62916249607184,Good
2,0x1604,CUS_0xd40,March,Aaron Maashoh,-500,821-00-0265,Scientist,19114.12,NaN,3,...,Good,809.98,28.609352,22 Years and 3 Months,No,49.574949,81.699521264648,Low_spent_Medium_value_payments,331.2098628537912,Good
3,0x1605,CUS_0xd40,April,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,NaN,3,...,Good,809.98,31.377862,22 Years and 4 Months,No,49.574949,199.4580743910713,Low_spent_Small_value_payments,223.45130972736786,Good
4,0x1606,CUS_0xd40,May,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,1824.843333,3,...,Good,809.98,24.797347,22 Years and 5 Months,No,49.574949,41.420153086217326,High_spent_Medium_value_payments,341.48923103222177,Good


In [6]:
df.dtypes

id                           object
customer_id                  object
month                        object
name                         object
age                          object
ssn                          object
occupation                   object
annual_income                object
monthly_inhand_salary       float64
num_bank_accounts             int64
num_credit_card               int64
interest_rate                 int64
num_of_loan                  object
type_of_loan                 object
delay_from_due_date           int64
num_of_delayed_payment       object
changed_credit_limit         object
num_credit_inquiries        float64
credit_mix                   object
outstanding_debt             object
credit_utilization_ratio    float64
credit_history_age           object
payment_of_min_amount        object
total_emi_per_month         float64
amount_invested_monthly      object
payment_behaviour            object
monthly_balance              object
credit_score                

In [7]:
df.isnull().sum()

id                              0
customer_id                     0
month                           0
name                         9985
age                             0
ssn                             0
occupation                      0
annual_income                   0
monthly_inhand_salary       15002
num_bank_accounts               0
num_credit_card                 0
interest_rate                   0
num_of_loan                     0
type_of_loan                11408
delay_from_due_date             0
num_of_delayed_payment       7002
changed_credit_limit            0
num_credit_inquiries         1965
credit_mix                      0
outstanding_debt                0
credit_utilization_ratio        0
credit_history_age           9030
payment_of_min_amount           0
total_emi_per_month             0
amount_invested_monthly      4479
payment_behaviour               0
monthly_balance              1200
credit_score                    0
dtype: int64

## Dealing with missing values and problematic types

### month
Let's add numeric column with month number, so sorting data in the future would be easy

In [8]:
df.month.unique()

array(['January', 'February', 'March', 'April', 'May', 'June', 'July',
       'August'], dtype=object)

In [9]:
months_mapping = {
    'January': 1,
    'February': 2,
    'March': 3,
    'April': 4,
    'May': 5,
    'June': 6,
    'July': 7,
    'August': 8,
    'September': 9,
    'October': 10,
    'November': 11,
    'December': 12
}

In [10]:
df['month_number'] = df.month.map(months_mapping)
df['month_number'].head(10)

0    1
1    2
2    3
3    4
4    5
5    6
6    7
7    8
8    1
9    2
Name: month_number, dtype: int64

### name column
All customers with same id have the same name. So let's try to fill empty names using customer_id

In [11]:
print(f"Initiall number of empty names: {df[['name']].isna().sum().values[0]}")

Initiall number of empty names: 9985


In [12]:
df['name'] = df[['customer_id', 'name']].groupby('customer_id').fillna(method='bfill')
df['name'] = df[['customer_id', 'name']].groupby('customer_id').fillna(method='ffill')
print(f'Number of empty names after fillna: {df["name"].isna().sum()}')

/var/folders/9z/rhfc541d14q5klxp9lqvgx8m0000gq/T/ipykernel_94310/3461443390.py:1: FutureWarning: DataFrameGroupBy.fillna is deprecated and will be removed in a future version. Use obj.ffill() or obj.bfill() for forward or backward filling instead. If you want to fill with a single value, use DataFrame.fillna instead
  df['name'] = df[['customer_id', 'name']].groupby('customer_id').fillna(method='bfill')
/var/folders/9z/rhfc541d14q5klxp9lqvgx8m0000gq/T/ipykernel_94310/3461443390.py:1: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['name'] = df[['customer_id', 'name']].groupby('customer_id').fillna(method='bfill')
/var/folders/9z/rhfc541d14q5klxp9lqvgx8m0000gq/T/ipykernel_94310/3461443390.py:2: FutureWarning: DataFrameGroupBy.fillna is deprecated and will be removed in a future version. Use obj.ffill() or obj.bfill() for forward or backward filling instead. If you want to fill with a single valu

Number of empty names after fillna: 0


### monthly_inhand_salary
Now we want to fill monthly_inhand_salary column. 
* We assume that it depends on annual_income column. 
* Unfortunately annual_income columns is of object and not numeric.
* If we try to cast annual income to numeric type we get an error:

In [13]:
try:
    df['annual_income'] = df['annual_income'].apply(pd.to_numeric)
except Exception as e:
    print(e)

Unable to parse string "34847.84_" at position 0


Let's see what are those problematic values we are dealing with:

In [14]:
problematic_values = df[df['annual_income'].apply(pd.to_numeric, errors='coerce').isna()]['annual_income'].values
problematic_values

array(['34847.84_', '30689.89_', '35547.71_', ..., '37188.1_',
       '39628.99_', '39628.99_'], dtype=object)

It seems like all of those problematic values have "_" in the end. Let's check it

In [15]:
for val in problematic_values:
    assert '_' == val[-1]

Let's check that we can cast all of those values to float if we delete "_" in the end

In [16]:
for val in problematic_values:
    float(val[:-1])

So now let's write function that will cast annual_income column to numeric

In [17]:
def custom_to_numeric(value: str):
    if value[-1] == '_':
        return float(value[:-1])
    return float(value)

In [18]:
df['annual_income'] = df['annual_income'].apply(custom_to_numeric)
df['annual_income'].dtype

dtype('float64')

Let's now fill the values for monthly_inhand_salary column.  
Let's check how monthly_inhand_salary depend on annual_income. To do that let's see the proportion between those values

In [19]:
not_empty_montly_salary_index = df['monthly_inhand_salary'].dropna().index
proportion = df['annual_income'][not_empty_montly_salary_index] / df['monthly_inhand_salary'][not_empty_montly_salary_index]
proportion.describe()

count    84998.000000
mean        67.244578
std        846.095019
min          8.089821
25%         11.553886
50%         12.025936
75%         12.553691
max      54110.522117
dtype: float64

In [20]:
np.quantile(proportion, 0.1), np.quantile(proportion, 0.9)

(np.float64(10.884076704509429), np.float64(13.568605658582612))

I assume there are some extreme values of annual_income, because mean is 67, but as you can see most of the values are between 10 and 13 with median being around 12, which makes sence, because year has 12 months. So let's fill empty values of this column with **annual_income/12**

In [21]:
df['monthly_inhand_salary'] = df['monthly_inhand_salary'].fillna(df['annual_income']/12)

### num_of_loan
This column represents the number of loans taken from the bank.  
This column should be numerical, but it's an object. Let's fix it.

In [22]:
s = df['num_of_loan'].unique()
s.sort()
s

array(['-100', '0', '0_', '1', '100', '1001', '1002', '1006', '1008',
       '101', '1014', '1015', '1017', '1019', '1023', '1027_', '103',
       '1030', '1035', '1036', '1039', '1040', '1046', '1047', '1048',
       '1053', '1054', '1070', '1074', '1077', '1085', '1088', '1091',
       '1094', '1096', '1103', '1106', '1110', '1112', '1127', '1129',
       '1129_', '1131', '1131_', '1132_', '1135', '1137', '1150', '1151',
       '1152', '1154', '1159', '1160', '1171_', '1178', '1181', '1182',
       '1185_', '1187', '1189', '119', '1196', '1202', '1204', '1209',
       '1210', '1214', '1216', '1217', '1219_', '1222', '1225', '1225_',
       '1227', '1228', '123', '1236', '1241', '1257', '1259', '126',
       '1265', '1271', '1274', '1279', '1289', '1294', '1296', '1297',
       '1298', '1300', '1302', '1307', '1311_', '1312', '1313', '1318',
       '1319', '131_', '132', '1320', '1320_', '1329', '1340', '1345',
       '1347_', '1348', '1353', '1354', '1359', '136', '1363', '1365',
   

We once again see that some of the values have underscores in the end. We will cast this field to numeric using the same function

In [23]:
df['num_of_loan'] = df['num_of_loan'].apply(custom_to_numeric)
df['num_of_loan'].dtype

dtype('float64')

Also we see that num_of_loan can be negative, which is strange.   
Let's look at types of loans correspoding to num_of_loan == -100:

In [24]:
num_of_loans_check = df[df['num_of_loan'] == -100][['num_of_loan', 'customer_id', 'name']]
num_of_loans_check

,num_of_loan,customer_id,name
31,-100.0,CUS_0xb891,Jasond
34,-100.0,CUS_0x1cdb,Deepaa
39,-100.0,CUS_0x1cdb,Deepaa
53,-100.0,CUS_0x284a,Nadiaq
61,-100.0,CUS_0x5407,Annk
...,...,...,...
99877,-100.0,CUS_0x3855,Xolai
99901,-100.0,CUS_0x4986,Charles Abbotta
99902,-100.0,CUS_0x4986,Charles Abbotta
99969,-100.0,CUS_0xf16,Maria Sheahanb


Let's look at some of the clients with num_of_loan = -100:

In [25]:
df[df['customer_id'] == 'CUS_0x5407'][['month', 'type_of_loan', 'num_of_loan']]

,month,type_of_loan,num_of_loan
56,January,"Not Specified, Auto Loan, and Student Loan",3.0
57,February,"Not Specified, Auto Loan, and Student Loan",3.0
58,March,"Not Specified, Auto Loan, and Student Loan",3.0
59,April,"Not Specified, Auto Loan, and Student Loan",3.0
60,May,"Not Specified, Auto Loan, and Student Loan",3.0
61,June,"Not Specified, Auto Loan, and Student Loan",-100.0
62,July,"Not Specified, Auto Loan, and Student Loan",3.0
63,August,"Not Specified, Auto Loan, and Student Loan",3.0


In [26]:
df[df['customer_id'] == 'CUS_0x4157'][['month', 'type_of_loan', 'num_of_loan']]

,month,type_of_loan,num_of_loan
64,January,"Personal Loan, Debt Consolidation Loan, and Au...",-100.0
65,February,"Personal Loan, Debt Consolidation Loan, and Au...",3.0
66,March,"Personal Loan, Debt Consolidation Loan, and Au...",3.0
67,April,"Personal Loan, Debt Consolidation Loan, and Au...",3.0
68,May,"Personal Loan, Debt Consolidation Loan, and Au...",3.0
69,June,"Personal Loan, Debt Consolidation Loan, and Au...",3.0
70,July,"Personal Loan, Debt Consolidation Loan, and Au...",3.0
71,August,"Personal Loan, Debt Consolidation Loan, and Au...",3.0


We can see that in those cases -100 is invalid. So let's set NaN value instead of -100:

In [27]:
df.loc[df['num_of_loan'] == -100, 'num_of_loan'] = np.nan

Also let's look at other values of num_of_loan

In [28]:
df['num_of_loan'].value_counts(sort=True).head(20)

num_of_loan
3.0       15104
2.0       15032
4.0       14743
0.0       10930
1.0       10606
6.0        7803
7.0        7344
5.0        7197
9.0        3702
8.0        3191
1150.0        4
1228.0        3
1480.0        3
288.0         3
773.0         3
430.0         3
227.0         2
1131.0        2
192.0         2
284.0         2
Name: count, dtype: int64

This data looks suspicious. What person would have 1000+ loans? Let's look at normalisation of this data:

In [29]:
display(df['num_of_loan'].value_counts(normalize=True, sort=True).head(10))
print(f"% of num of loans from 0 to 9: {df['num_of_loan'].value_counts(normalize=True, sort=True).head(10).sum()}")

num_of_loan
3.0    0.157130
2.0    0.156381
4.0    0.153375
0.0    0.113707
1.0    0.110337
6.0    0.081176
7.0    0.076401
5.0    0.074872
9.0    0.038513
8.0    0.033197
Name: proportion, dtype: float64

% of num of loans from 0 to 9: 0.9950896758353793


We can see that over 99.5% of rows has number of loans < 10. So let's assume that number of loans > 10 is a mistake and let's set NaN value instead of any value greater than 9:

In [30]:
df.loc[df['num_of_loan'] > 9, 'num_of_loan'] = np.nan

To fill missing values let's just make num_of_loan equal to number of loans avaliable in the previous month. If there is no info about the previous months, we will take the next month.

In [31]:
df['num_of_loan'].isna().sum()

np.int64(4348)

In [32]:
df['num_of_loan'] = df[['customer_id', 'num_of_loan', 'month_number']].sort_values('month_number').groupby('customer_id').fillna(method='ffill').fillna(method='bfill').sort_index()['num_of_loan']

/var/folders/9z/rhfc541d14q5klxp9lqvgx8m0000gq/T/ipykernel_94310/3368487835.py:1: FutureWarning: DataFrameGroupBy.fillna is deprecated and will be removed in a future version. Use obj.ffill() or obj.bfill() for forward or backward filling instead. If you want to fill with a single value, use DataFrame.fillna instead
  df['num_of_loan'] = df[['customer_id', 'num_of_loan', 'month_number']].sort_values('month_number').groupby('customer_id').fillna(method='ffill').fillna(method='bfill').sort_index()['num_of_loan']
/var/folders/9z/rhfc541d14q5klxp9lqvgx8m0000gq/T/ipykernel_94310/3368487835.py:1: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['num_of_loan'] = df[['customer_id', 'num_of_loan', 'month_number']].sort_values('month_number').groupby('customer_id').fillna(method='ffill').fillna(method='bfill').sort_index()['num_of_loan']


In [33]:
df['num_of_loan'].isna().sum()

np.int64(0)

### type_of_loan

In [34]:
all_loans_strings = list(df['type_of_loan'].dropna().unique())
all_loans_strings[:10]

['Auto Loan, Credit-Builder Loan, Personal Loan, and Home Equity Loan',
 'Credit-Builder Loan',
 'Auto Loan, Auto Loan, and Not Specified',
 'Not Specified',
 'Credit-Builder Loan, and Mortgage Loan',
 'Not Specified, Auto Loan, and Student Loan',
 'Personal Loan, Debt Consolidation Loan, and Auto Loan',
 'Not Specified, and Payday Loan',
 'Credit-Builder Loan, Personal Loan, and Auto Loan',
 'Payday Loan, and Payday Loan']

We can see that values are really different, but we are interested in all unique kinds of types:

In [35]:
unique_types = set()
for string_ in all_loans_strings:
    for type_ in string_.split(','):
        n_type = type_.replace(" and ","").strip()
        unique_types.add(n_type)
print(unique_types)

{'Debt Consolidation Loan', 'Payday Loan', 'Credit-Builder Loan', 'Auto Loan', 'Home Equity Loan', 'Personal Loan', 'Not Specified', 'Mortgage Loan', 'Student Loan'}


Let's look closer at this column, that represents the types of loan taken by a person. I assume we can fill empty values by using previuos data from the same client.



In [36]:
clients_with_nan_loans = list(df[df['type_of_loan'].isna()]['customer_id'].unique())
clients_with_nan_loans[:5]

['CUS_0x1cdb', 'CUS_0x95ee', 'CUS_0x6070', 'CUS_0x3553', 'CUS_0x132f']

In [37]:
df[df['customer_id'] == clients_with_nan_loans[0]][['type_of_loan', 'num_of_loan']]

,type_of_loan,num_of_loan
32,NaN,0.0
33,NaN,0.0
34,NaN,0.0
35,NaN,0.0
36,NaN,0.0
37,NaN,0.0
38,NaN,0.0
39,NaN,0.0


In [38]:
df[df['customer_id'] == clients_with_nan_loans[-2]][['type_of_loan', 'num_of_loan']]

,type_of_loan,num_of_loan
99904,NaN,0.0
99905,NaN,0.0
99906,NaN,0.0
99907,NaN,0.0
99908,NaN,0.0
99909,NaN,0.0
99910,NaN,0.0
99911,NaN,0.0


It seems like type_of_loan is NaN if num_of_loan == 0. Let's check that:

In [39]:
customers_groups = df.groupby('customer_id')
counter = 0
for cid in clients_with_nan_loans:
    group = customers_groups.get_group(cid)
    if not (all(group['type_of_loan'].fillna(0.0) == group['num_of_loan'])):
        counter += 1
        if counter == 30:
            display(group[['customer_id', 'type_of_loan', 'num_of_loan', 'month_number']])
print(counter)

/var/folders/9z/rhfc541d14q5klxp9lqvgx8m0000gq/T/ipykernel_94310/1229599172.py:5: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  if not (all(group['type_of_loan'].fillna(0.0) == group['num_of_loan'])):


,customer_id,type_of_loan,num_of_loan,month_number
57504,CUS_0xc4e7,NaN,3.0,1
57505,CUS_0xc4e7,NaN,0.0,2
57506,CUS_0xc4e7,NaN,0.0,3
57507,CUS_0xc4e7,NaN,0.0,4
57508,CUS_0xc4e7,NaN,0.0,5
57509,CUS_0xc4e7,NaN,0.0,6
57510,CUS_0xc4e7,NaN,0.0,7
57511,CUS_0xc4e7,NaN,0.0,8


59


/var/folders/9z/rhfc541d14q5klxp9lqvgx8m0000gq/T/ipykernel_94310/1229599172.py:5: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  if not (all(group['type_of_loan'].fillna(0.0) == group['num_of_loan'])):


In [40]:
df.type_of_loan.isna().sum()

np.int64(11408)

As we can see it's possible that type_of_loan is NaN, but num_of_loan is reasonable values. 
In this case we will fill type_of_loan "Not Specified" value, and otherwise we will create new value "No loans"

In [48]:
no_loans_index = df[df['num_of_loan'] == 0.0].index
no_loans_index

Index([   32,    33,    34,    35,    36,    37,    38,    39,    40,    41,
       ...
       99910, 99911, 99936, 99937, 99938, 99939, 99940, 99941, 99942, 99943],
      dtype='int64', length=11409)

In [51]:
df.loc[no_loans_index, ['type_of_loan']] = "No loans"

In [52]:
df[df['customer_id'] == clients_with_nan_loans[0]][['type_of_loan', 'num_of_loan']]

,type_of_loan,num_of_loan
32,No loans,0.0
33,No loans,0.0
34,No loans,0.0
35,No loans,0.0
36,No loans,0.0
37,No loans,0.0
38,No loans,0.0
39,No loans,0.0


In [59]:
still_nulls = df.type_of_loan.isna()
df[still_nulls][['customer_id', 'type_of_loan', 'num_of_loan', 'month_number']]

,customer_id,type_of_loan,num_of_loan,month_number
712,CUS_0x3861,NaN,2.0,1
2496,CUS_0x79b,NaN,1.0,1
2984,CUS_0xa560,NaN,3.0,1
6832,CUS_0x6b15,NaN,3.0,1
6880,CUS_0x29d2,NaN,4.0,1
...,...,...,...,...
89232,CUS_0xc317,NaN,6.0,1
89432,CUS_0x58b2,NaN,4.0,1
89640,CUS_0x2b3c,NaN,3.0,1
92208,CUS_0x30c1,NaN,3.0,1


In [60]:
df.type_of_loan = df.type_of_loan.fillna('Not Specified')
df[still_nulls][['customer_id', 'type_of_loan', 'num_of_loan', 'month_number']]

,customer_id,type_of_loan,num_of_loan,month_number
712,CUS_0x3861,Not Specified,2.0,1
2496,CUS_0x79b,Not Specified,1.0,1
2984,CUS_0xa560,Not Specified,3.0,1
6832,CUS_0x6b15,Not Specified,3.0,1
6880,CUS_0x29d2,Not Specified,4.0,1
...,...,...,...,...
89232,CUS_0xc317,Not Specified,6.0,1
89432,CUS_0x58b2,Not Specified,4.0,1
89640,CUS_0x2b3c,Not Specified,3.0,1
92208,CUS_0x30c1,Not Specified,3.0,1


### credit_history_age
Represents the age of credit history of the person

In [77]:
df.credit_history_age.head()

0    22 Years and 1 Months
1                      NaN
2    22 Years and 3 Months
3    22 Years and 4 Months
4    22 Years and 5 Months
Name: credit_history_age, dtype: object

In [78]:
print(f"Empty values count: {df.credit_history_age.isna().sum()}")

Empty values count: 9030


In [79]:
df.credit_history_age.value_counts()

credit_history_age
15 Years and 11 Months    446
19 Years and 4 Months     445
19 Years and 5 Months     444
17 Years and 11 Months    443
19 Years and 3 Months     441
                         ... 
0 Years and 3 Months       20
0 Years and 2 Months       15
33 Years and 7 Months      14
33 Years and 8 Months      12
0 Years and 1 Months        2
Name: count, Length: 404, dtype: int64

Let's turn this column into number of months as integer value:

In [85]:
a = np.nan
np.isnan(a)

np.True_

In [88]:
def credit_history_age_to_int(value: str):
    if pd.isnull(value):
        return value
    years_str, months_str = value.split(' and ')
    years = int(years_str.split(' ')[0])
    months = int(months_str.split(' ')[0])
    return 12 * years + months

In [90]:
df['credit_history_age'] = df['credit_history_age'].apply(credit_history_age_to_int)

Let's also fill empty values using customer info we gather from other rows:

In [99]:
df[df['credit_history_age'].isna()][['customer_id', 'credit_history_age']]

,customer_id,credit_history_age
1,CUS_0xd40,NaN
7,CUS_0xd40,NaN
19,CUS_0x2dbc,NaN
40,CUS_0x95ee,NaN
42,CUS_0x95ee,NaN
...,...,...
99944,CUS_0x51b3,NaN
99963,CUS_0x372c,NaN
99968,CUS_0xf16,NaN
99975,CUS_0xf16,NaN


In [101]:
df[df['customer_id'] == 'CUS_0xd40'][['customer_id', 'month_number', 'credit_history_age']]

,customer_id,month_number,credit_history_age
0,CUS_0xd40,1,265.0
1,CUS_0xd40,2,NaN
2,CUS_0xd40,3,267.0
3,CUS_0xd40,4,268.0
4,CUS_0xd40,5,269.0
5,CUS_0xd40,6,270.0
6,CUS_0xd40,7,271.0
7,CUS_0xd40,8,NaN
